In [1]:
import os
import re
from glob import glob
from datetime import datetime
import gzip

import pyart

import pandas as pd
import numpy as np
import pickle


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
def get_time_from_filename(filename, date):
    '''
    GET_TIME_FROM_FILENAME
    Capture the time string inside the filename and returns it.
    '''

    # Looking for date followed by underscore and 6 consecutives number (i.e.
    # the time)
    dtstr = re.findall(date + '.[0-9]{6}', filename)
    dtime = datetime.strptime(dtstr[0], '%Y%m%d-%H%M%S')
    return dtime


def find_nearest_filename(file_list, target_dt):
    dt_list = []
    for file in file_list:
        basename = os.path.basename(file)
        dt_list.append(get_time_from_filename(basename, target_dt.strftime('%Y%m%d')))
    dt_array = np.array(dt_list)
    time_diff = np.abs(dt_array-target_dt)
    min_idx = np.argmin(time_diff)
    min_ffn = file_list[min_idx]
    print('min time difference is', time_diff[min_idx],'for', os.path.basename(min_ffn))
    return min_ffn



def get_filter(radar, dbz_name='total_power',
                      zdr_name='differential_reflectivity',
                      phidp_name='differential_phase',
                      width_name='spectrum_width',
                      rhohv_name='cross_correlation_ratio'):
    
    """
    Loads a gaussian mixture model (pre generated) for filtering of clutter from radar volume
    This function was developed by Valentin Louf
    returns a gatefilter dictionary for masking radar data in the pyart radar object
    """
    
    # Load Scikit model
    with gzip.GzipFile(PATH_TO_GM_MODEL, 'r') as gzid:
        meteorological_echoes_GMM = pickle.load(gzid)
    
    df_orig = pd.DataFrame({'total_power': radar.fields[dbz_name]['data'].flatten(),
                            'differential_reflectivity': radar.fields[zdr_name]['data'].flatten(),
                            'differential_phase': radar.fields[phidp_name]['data'].flatten(),
                            'spectrum_width': radar.fields[width_name]['data'].flatten(),
                            'cross_correlation_ratio': radar.fields[rhohv_name]['data'].flatten(),
                           })

    df = df_orig.dropna()

    pos_droped = df_orig.dropna().index
    radar_cluster = meteorological_echoes_GMM.predict(df_orig.dropna())

    r = radar.range['data']
    time = radar.time['data']
    R, T = np.meshgrid(r, time)

    clus = np.zeros_like(R.flatten())
    clus[pos_droped] = radar_cluster + 1
    cluster = clus.reshape(R.shape)

    meteorological_signal = ((cluster >= 5) | (cluster == 2)) 

    radar.add_field('good_mask', {'data': meteorological_signal})

    gf = pyart.filters.GateFilter(radar)
    gf.exclude_equal('good_mask', 0)
    gf = pyart.correct.despeckle_field(radar, dbz_name, gatefilter=gf)
    _ = radar.fields.pop('good_mask')

    return gf

In [3]:
# DAILY CONVERSION - See below for select volume conversion

dt_str_list = ['20190124']

opol_root = '/g/data/hj10/admin/opol/level_1/in2019_v01'
out_folder = '/g/data/kl02/jss548/temp/opol_request/'
PATH_TO_GM_MODEL = '/home/548/jss548/dev/projects/YMC/GM_model_radar_metechoes.pkl.gz'

for dt_str in dt_str_list:
    #generate dt, paths and listing
    opol_path = opol_root + '/' + dt_str
    opol_ffn_list = sorted(glob(opol_path + '/*.hdf'))
    
    for opol_ffn in opol_ffn_list:
        print(opol_ffn)
        #add gmm gate filter
        radar = pyart.aux_io.read_odim_h5(opol_ffn, file_field_names=True)
        gf = get_filter(radar, dbz_name='DBZH',
                          zdr_name='ZDR',
                          phidp_name='PHIDP',
                          width_name='WRAD',
                          rhohv_name='RHOHV')
        #apply gatefilter to reflectivity field
        refl = radar.fields['DBZH']['data']
        refl_ma = np.ma.masked_where(gf._gate_excluded, refl)
        radar.add_field_like('DBZH', 'DBZH_gmm', refl_ma)

    #     #remove other fields
    #     for k in list(radar.fields.keys()):
    #         if k not in ['DBZH', 'DBZH_gmm']:
    #             radar.fields.pop(k)

        #save to file
        output_ffn = out_folder + os.path.basename(opol_ffn)[:-4] + '.nc'
        pyart.io.write_cfradial(output_ffn, radar)
    
#     #plot
#     display = pyart.graph.RadarDisplay(radar)
#     display.plot_ppi('DBZH_gmm', 0)
#    break

print('complete')

/g/data/hj10/admin/opol/level_1/in2019_v01/20190124/9776HUB-PPIVol-20190124-000034-0000.hdf


/g/data/en0/jss548/miniconda3/envs/radar-dev/lib/python3.8/site-packages/pyart/aux_io/odim_h5.py:295: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  start_time = datetime.datetime.utcfromtimestamp(start_epoch)
/g/data/en0/jss548/miniconda3/envs/radar-dev/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.mixture.gaussian_mixture module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.mixture. Anything that cannot be imported from sklearn.mixture is now part of the private API.
  warnings.warn(message, FutureWarning)
/g/data/en0/jss548/miniconda3/envs/radar-dev/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.21.3 when using version 0.23.1. This mig

/g/data/hj10/admin/opol/level_1/in2019_v01/20190124/9776HUB-PPIVol-20190124-000606-0000.hdf


/g/data/en0/jss548/miniconda3/envs/radar-dev/lib/python3.8/site-packages/pyart/aux_io/odim_h5.py:295: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  start_time = datetime.datetime.utcfromtimestamp(start_epoch)
/g/data/en0/jss548/miniconda3/envs/radar-dev/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.21.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


/g/data/hj10/admin/opol/level_1/in2019_v01/20190124/9776HUB-PPIVol-20190124-000721-0000.hdf


...

/g/data/hj10/admin/opol/level_1/in2019_v01/20190124/9776HUB-PPIVol-20190124-235528-0000.hdf


/g/data/en0/jss548/miniconda3/envs/radar-dev/lib/python3.8/site-packages/pyart/aux_io/odim_h5.py:295: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  start_time = datetime.datetime.utcfromtimestamp(start_epoch)
/g/data/en0/jss548/miniconda3/envs/radar-dev/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.21.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


/g/data/hj10/admin/opol/level_1/in2019_v01/20190124/9776HUB-PPIVol-20190124-235643-0000.hdf


/g/data/en0/jss548/miniconda3/envs/radar-dev/lib/python3.8/site-packages/pyart/aux_io/odim_h5.py:295: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  start_time = datetime.datetime.utcfromtimestamp(start_epoch)
/g/data/en0/jss548/miniconda3/envs/radar-dev/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.21.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


complete


In [ ]:
"""
year  m   d   h  m   s      lon         lat 
2019-01-23 09:12:26   143.87    -57.41    
2019-01-24 03:33:06   143.24    -59.44    
2019-01-24 08:19:46   142.95    -60.31    
2019-01-25 04:16:30   142.08    -62.86  
2019-01-25 07:27:11   141.92    -63.28    
2019-02-07 00:55:06   145.78    -64.20   
2019-02-07 02:30:22   145.69    -64.41  
2019-02-08 01:38:06   145.18    -65.47    
2019-02-10 23:02:12   152.33    -64.07    
2019-02-13 23:35:31   148.11    -65.62    
2019-02-15 23:26:05   149.65    -65.22     
"""

dt_str_list = ['2019-01-23 09:12:26', '2019-01-24 03:33:06', '2019-01-24 08:19:46',
              '2019-01-25 04:16:30', '2019-01-25 07:27:11', '2019-02-07 00:55:06',
              '2019-02-07 02:30:22', '2019-02-08 01:38:06', '2019-02-10 23:02:12',
              '2019-02-13 23:35:31', '2019-02-15 23:26:05']

opol_root = '/g/data/hj10/admin/opol/level_1/in2019_v01'
out_folder = '/g/data/kl02/jss548/temp/opol_request/'
PATH_TO_GM_MODEL = '/home/548/jss548/dev/projects/YMC/GM_model_radar_metechoes.pkl.gz'

for dt_str in dt_str_list:
    #generate dt, paths and listing
    target_dt = datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S')
    opol_path = opol_root + '/' + target_dt.strftime('%Y%m%d')
    opol_ffn_list = sorted(glob(opol_path + '/*.hdf'))
    #find nearest file
    target_opol_ffn = find_nearest_filename(opol_ffn_list, target_dt)
    #add gmm gate filter
    radar = pyart.aux_io.read_odim_h5(target_opol_ffn, file_field_names=True)
    gf = get_filter(radar, dbz_name='DBZH',
                      zdr_name='ZDR',
                      phidp_name='PHIDP',
                      width_name='WRAD',
                      rhohv_name='RHOHV')
    #apply gatefilter to reflectivity field
    refl = radar.fields['DBZH']['data']
    refl_ma = np.ma.masked_where(gf._gate_excluded, refl)
    radar.add_field_like('DBZH', 'DBZH_gmm', refl_ma)
    
#     #remove other fields
#     for k in list(radar.fields.keys()):
#         if k not in ['DBZH', 'DBZH_gmm']:
#             radar.fields.pop(k)
    
    #save to file
    output_ffn = out_folder + os.path.basename(target_opol_ffn)[:-4] + '.nc'
    pyart.io.write_cfradial(output_ffn, radar)
    
#     #plot
#     display = pyart.graph.RadarDisplay(radar)
#     display.plot_ppi('DBZH_gmm', 0)
#    break

print('complete')